In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('../data/processed/processed.csv')

In [3]:
df.shape

(5084, 6)

In [4]:
df.sample(3)

,brand,model_year,kms_driven,owner,location,price
1186,Yamaha,2019,9400.0,1st,New Delhi,85000
478,Bajaj,2017,33500.0,1st,other,40000
3275,Hero,2017,52818.0,1st,Delhi,49000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5084 entries, 0 to 5083
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   brand       5084 non-null   object 
 1   model_year  5084 non-null   int64  
 2   kms_driven  5084 non-null   float64
 3   owner       5084 non-null   object 
 4   location    5084 non-null   object 
 5   price       5084 non-null   int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 238.4+ KB


In [6]:
# scale price value
df['price'] = df['price'].apply(np.log1p)

## Category Encoding

In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder,MinMaxScaler


In [8]:
owner_categories = list(df.owner.unique()[::-1])
owner_categories

['5th', '4th', '2nd', '3rd', '1st']

In [9]:
df.sample()

,brand,model_year,kms_driven,owner,location,price
3181,Bajaj,2013,20071.0,1st,Delhi,10.5187


In [10]:
# ## Change the owner category to numerical encoding
# owners_ranks = {
#     '1st': 5,
#     '2nd': 4,
#     '3rd': 3,
#     '4th': 2,
#     '5th': 1,
    
# }

# df['owner'] = df['owner'].apply(lambda x: owners_ranks.get(x,1))

# df.info()

In [11]:
df.sample(3)

,brand,model_year,kms_driven,owner,location,price
5024,Bajaj,2020,66000.0,3rd,Jaipur,10.905974
2586,Honda,2018,30108.0,1st,Delhi,11.034906
4499,other,2017,3300.0,1st,Bangalore,13.527830


In [12]:
# ['5th','4th','3rd','2nd','1st']

In [13]:
from sklearn.base import BaseEstimator, TransformerMixin
# Custom transformer to normalize model_year
class ModelYearTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        super().__init__()

    def fit(self,X,y = None):
        return self

    def transform(self,X,y=None):
        X_ = X.copy()
        X_.model_year = X_.model_year - 2010

        return X_



category_transformer = ColumnTransformer([
    ("brand_ohe",OneHotEncoder(dtype=np.int16,sparse=False,handle_unknown='ignore'),[0]),
    ("model_year_normalizer",ModelYearTransformer(),[1]),
    ("kms_driven_min_max_scaler",MinMaxScaler(),[2]),
    ("owner_ordinal_enc",OrdinalEncoder(categories='auto',handle_unknown='ignore',dtype=np.int16),[3]),
    ("location_ohe",OneHotEncoder(dtype=np.int16, sparse=False,handle_unknown='ignore'),[4]),
],remainder='passthrough')



In [14]:
# Scale the price data
# y = MinMaxScaler().fit_transform([y])[0]

In [15]:
X = df.drop('price',axis=1)
y = df['price']

In [16]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=.2,random_state=1234)

## Model Building

In [17]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

from sklearn import set_config

set_config(display='diagram')

In [18]:

def build_pipeline_with_estimator(estimator):
    return Pipeline([
    ('category_transformer',category_transformer),
    ('estimator',estimator),
])


In [19]:
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
from sklearn.model_selection import cross_val_score
# MAPE
def mape(targets, predictions):
    return np.mean(np.abs((targets - predictions)) / targets) * 100

# Adjusted R^2
def adj_r2(ind_vars, targets, predictions):
    r2 = r2_score(targets, predictions)
    n = ind_vars.shape[0]
    k = ind_vars.shape[1]
    return 1-((1-r2)*(n-1)/(n-k-1))

# Model performance check
def model_perf(model, inp, out):

    y_pred = model.predict(inp)
    y_act = out.values

    cross_val = cross_val_score(model, inp,out,cv=10)
    

    return pd.DataFrame({
                "RMSE": np.sqrt(mean_squared_error(y_act, y_pred)),
                "MAE": mean_absolute_error(y_act, y_pred),
                "MAPE": mape(y_act, y_pred),
                "R^2": r2_score(y_act, y_pred),
                "Adjusted R^2": adj_r2(inp, y_act, y_pred),
                "Cross Val Score (Mean)": cross_val.mean()
           }, index=[0])

## LinearRegression

In [20]:
liner_regressor = build_pipeline_with_estimator(LinearRegression())

params = {}

liner_regressor = GridSearchCV(liner_regressor,params,cv=10)

liner_regressor.fit(X_train,y_train)

print('Linear Regression Train Performance.\n')
print(model_perf(liner_regressor,X_train,y_train))

print('Linear Regression Test Performance.\n')
print(model_perf(liner_regressor,X_test,y_test))

Linear Regression Train Performance.

       RMSE       MAE      MAPE       R^2  Adjusted R^2  \
0  0.497377  0.354853  3.315087  0.571515      0.570988   

   Cross Val Score (Mean)  
0                0.559561  
Linear Regression Test Performance.

       RMSE       MAE     MAPE       R^2  Adjusted R^2  Cross Val Score (Mean)
0  0.484819  0.341853  3.20454  0.583979      0.581921                0.567064


## RandomForest

In [21]:
from sklearn.ensemble import RandomForestRegressor

model = build_pipeline_with_estimator(RandomForestRegressor())

# params = {
#     'estimator__criterion': ['gini','entropy'],
#     'estimator__n_estimators': [100],
#     'estimator__max_depth': [5,10,15,20,25],
#     'estimator__min_samples_split': range(2,11),
#     'estimator__max_features': ['auto','sqrt','log2'],
# }

# model = GridSearchCV(model,params,cv=10)

model.fit(X_train,y_train)

print('RandomForest Train Performance.\n')
print(model_perf(model,X_train,y_train))

print('RandomForest Test Performance.\n')
print(model_perf(model,X_test,y_test))

RandomForest Train Performance.

       RMSE      MAE      MAPE       R^2  Adjusted R^2  Cross Val Score (Mean)
0  0.191249  0.12293  1.142833  0.936648       0.93657                0.623409
RandomForest Test Performance.

       RMSE       MAE      MAPE       R^2  Adjusted R^2  \
0  0.452873  0.293273  2.727927  0.636998      0.635203   

   Cross Val Score (Mean)  
0                0.549022  


## XGBoost

In [22]:
from xgboost import XGBRegressor

xgboost = build_pipeline_with_estimator(XGBRegressor())

xgboost.fit(X_train,y_train)

print('xgboost Train Performance.\n')
print(model_perf(xgboost,X_train,y_train))

print('xgboost Test Performance.\n')
print(model_perf(xgboost,X_test,y_test))

xgboost Train Performance.

       RMSE       MAE      MAPE      R^2  Adjusted R^2  Cross Val Score (Mean)
0  0.261629  0.180147  1.669747  0.88144      0.881294                0.655665
xgboost Test Performance.

       RMSE      MAE      MAPE       R^2  Adjusted R^2  Cross Val Score (Mean)
0  0.450211  0.29155  2.715033  0.641252      0.639478                 0.55722


In [23]:
from sklearn.neighbors import KNeighborsRegressor

model = build_pipeline_with_estimator(KNeighborsRegressor())

params = {
    'estimator__n_neighbors': [3,5,7,9,12],
    'estimator__algorithm': ['ball_tree','kd_tree','brute'],
    'estimator__leaf_size': [5,10,30,32,35,40,50]
}

model = GridSearchCV(model,params,cv=10)

model.fit(X_train,y_train)

print('KNeighborsRegressor Train Performance.\n')
print(model_perf(model,X_train,y_train))

print('KNeighborsRegressor Test Performance.\n')
print(model_perf(model,X_test,y_test))